In [1]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
from qdrant_client import QdrantClient, models

In [5]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [2]:
pip install fastembed

Note: you may need to restart the kernel to use updated packages.


In [3]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [18]:
#Q1:
from fastembed import TextEmbedding

# Initialize the embedding model (default is "BAAI/bge-small-en-v1.5")
embedding_model = TextEmbedding("jinaai/jina-embeddings-v2-small-en")

# Your query to embed
query = ["I just discovered the course. Can I join now?"]

# Generate embeddings (returns a generator)
embeddings_generator_q1 = embedding_model.embed(query)

# Convert generator to list (each element is a numpy array)
embeded_query_q1 = list(embeddings_generator_q1)[0]

# embeddings_list[0] is the numpy array embedding for the query
print("Embeded vector shape:", embeded_query_q1.shape)

# Minimal value in the embedding vector
print("Minimal value in embeded vector:", embeded_query_q1.min())

Embeded vector shape: (512,)
Minimal value in embeded vector: -0.11726373885183883


In [23]:
#Q2
doc = ['Can I still join the course after the start date?']
embeddings_generator_q2 = embedding_model.embed(doc)
embeded_query_q2 = list(embeddings_generator_q2)[0]

embeded_query_q2.dot(embeded_query_q1)
#embeded_query_q1.dot(embeded_query_q2)

np.float64(0.9008528895674548)